In [11]:
import sys 
print(sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [25]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gkg99\AppData\Roaming\nltk_data...


True

In [12]:
# python version 3.8.6rc1
import pandas as pd
import string
import util
import io
import os
from pprint import pprint
#import spacy
import pickle
import re

import warnings
warnings.filterwarnings(action='once')
#warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gkg99\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
comments = pd.read_csv("train.csv")
comments.head()

#(comments.iloc[:, 2:].sum())
# merging different kinds of harmful indicators
comments['harmful'] = comments.iloc[:, 2:].sum(axis=1)
#comments[comments['harmful']!=0] = 1
df = comments[['comment_text',"harmful"]].copy()
#selected.dtypes

df_harmful = df[df['harmful'] > 0] 
df_not_harmful = df[df['harmful'] == 0] 
len(df),df.head()

(159571,
                                         comment_text  harmful
 0  Explanation\nWhy the edits made under my usern...        0
 1  D'aww! He matches this background colour I'm s...        0
 2  Hey man, I'm really not trying to edit war. It...        0
 3  "\nMore\nI can't make any real suggestions on ...        0
 4  You, sir, are my hero. Any chance you remember...        0)

In [20]:
len(df_harmful),df_harmful.head()

(16225,
                                          comment_text  harmful
 6        COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK        4
 12  Hey... what is it..\n@ | talk .\nWhat is it......        1
 16  Bye! \n\nDon't look, come or think of comming ...        1
 42  You are gay or antisemmitian? \n\nArchangel WH...        4
 43           FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!        3)

In [23]:
def prepossess(df):
    def remove_punctuation(text):
        #return str(text).translate(str.maketrans('', '', string.punctuation))
        text_nopunct = "".join([char for char in str(text) if char not in string.punctuation])
        return text_nopunct

    def tokenize_text(text):
        tokens = nltk.word_tokenize(text) 
        tokens = [token.strip() for token in tokens]
        return tokens

    def remove_stopwords(tokens):
        stopword_list = nltk.corpus.stopwords.words('english')
        filtered_tokens = [token for token in tokens if token not in stopword_list]
        #filtered_text = ' '.join(filtered_tokens)    
        return filtered_tokens

    def expand_contractions(text, contraction_mapping):

        contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                          flags=re.IGNORECASE|re.DOTALL)
        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match)\
                                    if contraction_mapping.get(match)\
                                    else contraction_mapping.get(match.lower())                       
            expanded_contraction = first_char+expanded_contraction[1:]
            return expanded_contraction

        expanded_text = contractions_pattern.sub(expand_match, text)
        expanded_text = re.sub("'", "", expanded_text)
        return expanded_text

    def stemming(tokenized_text):
        ps = nltk.PorterStemmer()
        stemmed = [ps.stem(word) for word in tokenized_text]
        return stemmed

    def lemmatize(tokenized_text):
        wn = nltk.WordNetLemmatizer()
        lemmatized = [wn.lemmatize(word) for word in tokenized_text]
        return lemmatized
    
    df["comment_text_clean"] = df["comment_text"].apply(lambda x: remove_punctuation(x))
    df["comment_text_clean"] = df["comment_text_clean"].apply(lambda x: expand_contractions(x.lower(), util.contraction_mapping))
    df["tokenized"] = df["comment_text_clean"].apply(lambda x: tokenize_text(x))
    df['tokenized'] =  df["tokenized"].apply(lambda x: remove_stopwords(x))
    df['stemmed'] =  df["tokenized"].apply(lambda x: stemming(x))
    df['lemmatized'] =  df["stemmed"].apply(lambda x: lemmatize(x))
    return df

df = prepossess(df)
df.head()



RecursionError: maximum recursion depth exceeded in comparison

In [26]:
df_harmful = prepossess(df_harmful)
df_harmful.head()


<ipython-input-23-861b4023ff95>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_text_clean"] = df["comment_text"].apply(lambda x: remove_punctuation(x))
<ipython-input-23-861b4023ff95>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemmatized'] =  df["stemmed"].apply(lambda x: lemmatize(x))


,comment_text,harmful,comment_text_clean,tokenized,stemmed,lemmatized
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,4,cocksucker before you piss around on my work,"[cocksucker, piss, around, work]","[cocksuck, piss, around, work]","[cocksuck, piss, around, work]"
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,hey what is it\n talk \nwhat is it an exclusi...,"[hey, talk, exclusive, group, wp, talibanswho,...","[hey, talk, exclus, group, wp, talibanswho, go...","[hey, talk, exclus, group, wp, talibanswho, go..."
16,"Bye! \n\nDon't look, come or think of comming ...",1,bye \n\ndont look come or think of comming bac...,"[bye, dont, look, come, think, comming, back, ...","[bye, dont, look, come, think, com, back, tosser]","[bye, dont, look, come, think, com, back, tosser]"
42,You are gay or antisemmitian? \n\nArchangel WH...,4,you are gay or antisemmitian \n\narchangel whi...,"[gay, antisemmitian, archangel, white, tiger, ...","[gay, antisemmitian, archangel, white, tiger, ...","[gay, antisemmitian, archangel, white, tiger, ..."
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",3,fuck your filthy mother in the ass dry,"[fuck, filthy, mother, ass, dry]","[fuck, filthi, mother, ass, dri]","[fuck, filthi, mother, as, dri]"


In [27]:
def token_to_corpus(token):
    data_words = token.values.tolist()
    #print(data_words[:1][0][:30])
    id2word = corpora.Dictionary(data_words)
    # Create Corpus
    texts = data_words
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]  
    return corpus, id2word

def generate_pyLDAvis(corpus, id2word,num_topics = 10, save = False):
    lda_model = LdaModel(corpus = corpus,
                     id2word = id2word,
                     num_topics = num_topics,
                     random_state = 0,
                     chunksize = 100,
                     alpha = "auto",
                     per_word_topics = True,
    )
    LDA_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    if(save):
        pyLDAvis.save_html(LDA_vis, "./results/pyLDAvis.html")
    pprint(lda_model.print_topics())
    return LDA_vis




In [28]:
simple_corpus, simple_id2word = token_to_corpus(df_harmful['lemmatized'][:100])
simple_LDA_vis = generate_pyLDAvis(simple_corpus, simple_id2word)
pyLDAvis.enable_notebook()
simple_LDA_vis

[(0,
  '0.026*"fuck" + 0.011*"edit" + 0.008*"mother" + 0.008*"would" + 0.007*"god" '
  '+ 0.007*"know" + 0.006*"refer" + 0.006*"dead" + 0.006*"wikipedia" + '
  '0.006*"made"'),
 (1,
  '0.014*"fuck" + 0.014*"go" + 0.012*"like" + 0.012*"peopl" + 0.009*"edit" + '
  '0.009*"wikipedia" + 0.008*"cur" + 0.007*"shit" + 0.007*"right" + '
  '0.006*"fire"'),
 (2,
  '0.025*"fuck" + 0.021*"get" + 0.010*"daughter" + 0.009*"see" + 0.009*"dog" + '
  '0.009*"like" + 0.008*"son" + 0.008*"father" + 0.008*"got" + 0.008*"say"'),
 (3,
  '0.019*"fuck" + 0.019*"dont" + 0.011*"say" + 0.010*"go" + 0.008*"block" + '
  '0.008*"mother" + 0.007*"gay" + 0.007*"get" + 0.007*"daughter" + '
  '0.007*"suck"'),
 (4,
  '0.011*"pov" + 0.011*"articl" + 0.010*"capoeira" + 0.009*"histori" + '
  '0.007*"get" + 0.006*"ludim" + 0.006*"son" + 0.006*"take" + 0.006*"say" + '
  '0.006*"time"'),
 (5,
  '0.016*"as" + 0.014*"like" + 0.008*"life" + 0.008*"hardi" + 0.008*"littl" + '
  '0.008*"look" + 0.007*"sorri" + 0.007*"one" + 0.006*"

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.000325  0.102985       1        1  14.700707
7      0.037506 -0.111399       2        1  13.787125
9      0.097768  0.034859       3        1  13.005861
2     -0.079958 -0.005182       4        1  11.392826
6      0.098640  0.010851       5        1  10.938918
1      0.002212  0.026317       6        1   9.149227
4     -0.086663  0.032891       7        1   7.962143
3     -0.041268 -0.036764       8        1   7.549295
5     -0.005084 -0.036715       9        1   6.366204
0     -0.022827 -0.017843      10        1   5.147693, topic_info=       Term       Freq      Total Category  logprob  loglift
55     fuck  26.000000  26.000000  Default  30.0000  30.0000
664     fun   6.000000   6.000000  Default  29.0000  29.0000
92       as   4.000000   4.000000  Default  28.0000  28.0000
116     get  14.000000  14.000000  Default  27.0000  27.0000
195    like  14.000000  14.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
32     dont   0.626450  12.484158  Topic10  -5.2784  -0.0255
120  asshol   0.514372   3.870549  Topic10  -5.4756   0.9484
199    long   0.474767   2.564423  Topic10  -5.5557   1.2800
41   anymor   0.476330   2.733822  Topic10  -5.5524   1.2193
2      piss   0.449508   2.847793  Topic10  -5.6104   1.1205

[611 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
678       4  0.633415  0427
679       4  0.633385    20
680       4  0.633375  2004
638       6  0.762066  2005
256       5  0.734508  2007
...     ...       ...   ...
254       2  0.245510  your
254       3  0.245510  your
254       6  0.245510  your
255       2  0.258601  youv
255       6  0.258601  youv

[728 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 10, 3, 7, 2, 5, 4, 6, 1])

In [53]:
#BAG-OF-WORDS
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

from sklearn.feature_extraction.text import CountVectorizer

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

count_vect = CountVectorizer(analyzer=clean_text)
X_counts = count_vect.fit_transform(df_harmful['comment_text'])
print(X_counts.shape)
print(count_vect.get_feature_names_out())

(16225, 31176)
['' '0' '00' ... 'ｃｏｍ' 'ｗｗｗ' '𒁳']


In [54]:
X_counts_df = pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names_out())
X_counts_df.head(10)

,,0,00,000,00000000,0000001,0001,0003,0004,001,...,影武者,永訣,竜龙,臭妈的烂b,見学迷惑,連絡,ａｎｏｎｔａｌｋ,ｃｏｍ,ｗｗｗ,𒁳
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#N-grams
from sklearn.feature_extraction.text import CountVectorizer

ngram_vect = CountVectorizer(ngram_range=(2,2),analyzer=clean_text) # It applies only bigram vectorizer
X_counts = ngram_vect.fit_transform(df_harmful['comment_text'])
print(X_counts.shape)
print(ngram_vect.get_feature_names_out())

C:\Users\gkg99\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:544: UserWarning: The parameter 'ngram_range' will not be used since 'analyzer' is callable'
  warnings.warn(


(16225, 31176)
['' '0' '00' ... 'ｃｏｍ' 'ｗｗｗ' '𒁳']


In [45]:
X_counts_df = pd.DataFrame(X_counts.toarray(), columns=ngram_vect.get_feature_names_out())
X_counts_df.head(10)

,,0,00,000,00000000,0000001,0001,0003,0004,001,...,影武者,永訣,竜龙,臭妈的烂b,見学迷惑,連絡,ａｎｏｎｔａｌｋ,ｃｏｍ,ｗｗｗ,𒁳
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(df_harmful['comment_text'])
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names_out())

(16225, 31176)
['' '0' '00' ... 'ｃｏｍ' 'ｗｗｗ' '𒁳']


In [49]:
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names_out())
X_tfidf_df.head(10)

,,0,00,000,00000000,0000001,0001,0003,0004,001,...,影武者,永訣,竜龙,臭妈的烂b,見学迷惑,連絡,ａｎｏｎｔａｌｋ,ｃｏｍ,ｗｗｗ,𒁳
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
print(X_tfidf_df.loc[(X_tfidf_df!=0).any(axis=1)])

                   0   00  000  00000000  0000001  0001  0003  0004  001  ...  \
0      0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
1      0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
2      0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
3      0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
4      0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
...         ...  ...  ...  ...       ...      ...   ...   ...   ...  ...  ...   
16220  0.026342  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
16221  0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
16222  0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
16223  0.054299  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   
16224  0.000000  0.0  0.0  0.0       0.0      0.0   0.0   0.0   0.0  0.0  ...   

       影武者   永訣   竜龙  臭妈的烂b